In [1]:
%load_ext autoreload
%autoreload 2

# LangChain Expression Language (LCEL)

In [2]:
# LCEL to generate a joke

In [3]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template("tell me a short joke about {topic}")
model = ChatOpenAI(model="gpt-4")
output_parser = StrOutputParser()

chain = prompt | model | output_parser

chain.invoke({"topic": "ice cream"})

"Why don't ice creams ever get invited to parties?\n\nBecause they always bring a meltdown!"

In [4]:
prompt_value = prompt.invoke({"topic": "ice cream"})
prompt_value

ChatPromptValue(messages=[HumanMessage(content='tell me a short joke about ice cream')])

In [5]:
prompt_value.to_messages()

[HumanMessage(content='tell me a short joke about ice cream')]

In [6]:
prompt_value.to_string()

'Human: tell me a short joke about ice cream'

In [7]:
message = model.invoke(prompt_value)
message

AIMessage(content="Why don't ice creams ever get into arguments?\n\nBecause they always stay cool!")

In [8]:
from langchain_openai.llms import OpenAI

llm = OpenAI(model="gpt-3.5-turbo-instruct")
llm.invoke(prompt_value)

'Robot: Why did the ice cream go to therapy? Because it was feeling a little melted.'

In [9]:
output_parser.invoke(message)

"Why don't ice creams ever get into arguments?\n\nBecause they always stay cool!"

In [10]:
# if you want to break it down
input = {"topic": "ice cream"}

prompt.invoke(input)

ChatPromptValue(messages=[HumanMessage(content='tell me a short joke about ice cream')])

In [11]:
(prompt | model).invoke(input)

AIMessage(content="Why don't ice creams ever argue?\n\nBecause they always find a way to stay cool!")

## A more complicated RAG example

In [2]:
from langchain_community.vectorstores import DocArrayInMemorySearch
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_openai.chat_models import ChatOpenAI
from langchain_openai.embeddings import OpenAIEmbeddings

In [3]:
vectorstore = DocArrayInMemorySearch.from_texts(
    ["harrison worked at kensho", "bears like to eat honey"],
    embedding=OpenAIEmbeddings(),
)
retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
model = ChatOpenAI()
output_parser = StrOutputParser()

setup_and_retrieval = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
)
chain = setup_and_retrieval | prompt | model | output_parser

chain.invoke("where did harrison work?")

'Harrison worked at Kensho.'

In [4]:
retriever.invoke("where did harrison work?")

[Document(page_content='harrison worked at kensho'),
 Document(page_content='bears like to eat honey')]

In [7]:
test = model | output_parser

test.invoke("hai")

'Hello! How can I assist you today?'

In [8]:
# interface

model = ChatOpenAI()
prompt = ChatPromptTemplate.from_template("tell me a joke about {topic}")
chain = prompt | model

In [10]:
chain.input_schema.schema()

{'title': 'PromptInput',
 'type': 'object',
 'properties': {'topic': {'title': 'Topic', 'type': 'string'}}}

In [11]:
prompt.input_schema.schema()

{'title': 'PromptInput',
 'type': 'object',
 'properties': {'topic': {'title': 'Topic', 'type': 'string'}}}

In [12]:
model.input_schema.schema()

{'title': 'ChatOpenAIInput',
 'anyOf': [{'type': 'string'},
  {'$ref': '#/definitions/StringPromptValue'},
  {'$ref': '#/definitions/ChatPromptValueConcrete'},
  {'type': 'array',
   'items': {'anyOf': [{'$ref': '#/definitions/AIMessage'},
     {'$ref': '#/definitions/HumanMessage'},
     {'$ref': '#/definitions/ChatMessage'},
     {'$ref': '#/definitions/SystemMessage'},
     {'$ref': '#/definitions/FunctionMessage'},
     {'$ref': '#/definitions/ToolMessage'}]}}],
 'definitions': {'StringPromptValue': {'title': 'StringPromptValue',
   'description': 'String prompt value.',
   'type': 'object',
   'properties': {'text': {'title': 'Text', 'type': 'string'},
    'type': {'title': 'Type',
     'default': 'StringPromptValue',
     'enum': ['StringPromptValue'],
     'type': 'string'}},
   'required': ['text']},
  'AIMessage': {'title': 'AIMessage',
   'description': 'Message from an AI.',
   'type': 'object',
   'properties': {'content': {'title': 'Content',
     'anyOf': [{'type': 'strin

In [13]:
prompt.output_schema.schema()

{'title': 'ChatPromptTemplateOutput',
 'anyOf': [{'$ref': '#/definitions/StringPromptValue'},
  {'$ref': '#/definitions/ChatPromptValueConcrete'}],
 'definitions': {'StringPromptValue': {'title': 'StringPromptValue',
   'description': 'String prompt value.',
   'type': 'object',
   'properties': {'text': {'title': 'Text', 'type': 'string'},
    'type': {'title': 'Type',
     'default': 'StringPromptValue',
     'enum': ['StringPromptValue'],
     'type': 'string'}},
   'required': ['text']},
  'AIMessage': {'title': 'AIMessage',
   'description': 'Message from an AI.',
   'type': 'object',
   'properties': {'content': {'title': 'Content',
     'anyOf': [{'type': 'string'},
      {'type': 'array',
       'items': {'anyOf': [{'type': 'string'}, {'type': 'object'}]}}]},
    'additional_kwargs': {'title': 'Additional Kwargs', 'type': 'object'},
    'type': {'title': 'Type',
     'default': 'ai',
     'enum': ['ai'],
     'type': 'string'},
    'name': {'title': 'Name', 'type': 'string'},
 

In [14]:
chain.invoke({"topic": "bears"})

AIMessage(content='Why did the bear join the computer repair business? \n\nBecause he heard they had a lot of bytes!')

In [17]:
# batching works better with langchain now
chain.batch([{"topic": "bears"}]*10)

[AIMessage(content='Why did the bear bring a backpack to the picnic? \nIn case he wanted to go on a bear-y nice hike afterwards!'),
 AIMessage(content='Why did the bear dissolve in water?\nBecause it was polar-ic!'),
 AIMessage(content="Why did the bear break up with his girlfriend? \n\nBecause he couldn't bear the relationship anymore!"),
 AIMessage(content="Why don't bears like fast food? Because they can't catch it!"),
 AIMessage(content='Why did the bear bring a flashlight to the party? \n\nBecause he heard it was a "beary" good time!'),
 AIMessage(content='Why did the bear break up with his girlfriend?\n\nBecause she was unbearable!'),
 AIMessage(content="Why don't bears wear shoes?\n\nBecause they have bear feet!"),
 AIMessage(content="Why did the bear break up with his girlfriend? \n\nBecause he couldn't bear the relationship anymore!"),
 AIMessage(content="Why don't bears like fast food? Because they can't catch it!"),
 AIMessage(content="Why did the bear break up with his girl

## Understanding Runnables

let's try and experiment with runnable a bit more because that is the weakest link in the entire LCEL approach.

In [2]:
# RunnableParallel
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

vectorstore = FAISS.from_texts(
    ["harrison worked at kensho"], embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
model = ChatOpenAI()

retrieval_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

retrieval_chain.invoke("where did harrison work?")

'Harrison worked at Kensho.'

In [7]:
from langchain_core.runnables import RunnableParallel

dummy_chain = (
    RunnableParallel(context= retriever, question= RunnablePassthrough())
    | prompt
)

dummy_chain.invoke("where did harrison work?")

ChatPromptValue(messages=[HumanMessage(content="Answer the question based only on the following context:\n[Document(page_content='harrison worked at kensho')]\n\nQuestion: where did harrison work?\n")])

In [18]:
from operator import itemgetter

f = itemgetter("text")
f({"text": 1})

1

In [10]:
retriever.input_schema.schema()

{'title': 'VectorStoreRetrieverInput', 'type': 'string'}

In [11]:
retriever.invoke("test")

[Document(page_content='harrison worked at kensho')]

In [16]:
# you can also parallelize steps
model = ChatOpenAI()
joke_chain = ChatPromptTemplate.from_template("tell me a joke about {topic}") | model 
poem_chain = (
    ChatPromptTemplate.from_template("write a 2-line poem about {topic}") | model
)

map_chain = (RunnableParallel({
    "joke": joke_chain, 
    "poem":poem_chain
}) | {"joke": itemgetter("joke") | StrOutputParser(), "joke"}
            )

map_chain.invoke({"topic": "bear"})

{'joke': "Why don't bears like fast food?\n\nBecause they can't catch it!"}

In [17]:
# runnable pass through
runnable = RunnableParallel(
    passed=RunnablePassthrough(),
    extra=RunnablePassthrough.assign(mult=lambda x: x["num"] * 5),
    modified = lambda x: x["num"] + 1,
)
runnable.invoke({"num": 1})

{'passed': {'num': 1}, 'extra': {'num': 1, 'mult': 5}, 'modified': 2}

In [21]:
# runnable lambda
from langchain_core.runnables import RunnableLambda

def length_function(text):
    return len(text)


def _multiple_length_function(text1, text2):
    return len(text1) * len(text2)


def multiple_length_function(_dict):
    return _multiple_length_function(_dict["text1"], _dict["text2"])


prompt = ChatPromptTemplate.from_template("what is {a} + {b}")
model = ChatOpenAI()

chain1 = prompt | model

chain = (
    {
        "a": itemgetter("foo") | RunnableLambda(length_function),
        "b": {"text1": itemgetter("foo"), "text2": itemgetter("bar")}
        | RunnableLambda(multiple_length_function),
    }
    | prompt
    | model
)

chain.invoke({"foo": "bar", "bar": "gah"})

AIMessage(content='3 + 9 = 12')

In [23]:
# RunnableConfig
import json
from langchain_core.runnables import RunnableConfig

def parse_or_fix(text: str, config: RunnableConfig):
    fixing_chain = (
        ChatPromptTemplate.from_template(
            "Fix the following text:\n\n```text\n{input}\n```\nError: {error}"
            " Don't narrate, just respond with the fixed data."
        )
        | ChatOpenAI()
        | StrOutputParser()
    )
    for _ in range(3):
        try:
            return json.loads(text)
        except Exception as e:
            text = fixing_chain.invoke({"input": text, "error": e}, config)
    return "Failed to parse"

In [24]:
from langchain.callbacks import get_openai_callback

with get_openai_callback() as cb:
    output = RunnableLambda(parse_or_fix).invoke(
        "{foo: bar}", {"tags": ["my-tag"], "callbacks": [cb]}
    )
    print(output)
    print(cb)

{'foo': 'bar'}
Tokens Used: 62
	Prompt Tokens: 56
	Completion Tokens: 6
Successful Requests: 1
Total Cost (USD): $9.6e-05
